# Songify: An Apache Cassandra Keyspace


This notebook creates an Apache Cassandra keyspace `songifyks` for the music app, *Songify*. The data is modeled on the following 3 queries:

1. `artist`, `song` title and song's `length` for `sessionId=338`, and `itemInSession=4`

2. `artist`, `song` (sorted by `itemInSession`) and user (`firstName` and `lastName`) for `userid=10`, `sessionid=182`

3. every user name (`firstName` and `lastName`) for the song 'All Hands Against His Own'


In [ ]:
import numpy as np
import pandas as pd
import cassandra
import re
import os
import glob
import json
import csv
from pathlib import Path
from cassandra.cluster import Cluster

def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

<br>

List of paths to original event data partitioned by date.

In [ ]:

data_path = str(Path().home().parent) + '/seed_data'

for root, dirs, files in os.walk(data_path):
    file_path_list = glob.glob(os.path.join(root, '*'))

<br>

Create single csv data file from event data to populate Apache Casssandra tables.

In [ ]:
full_data_rows_list = [] 
    
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        for line in csvreader:
            full_data_rows_list.append(line) 


csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName',
                     'length', 'level','location','sessionId','song','userId'])

    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6],
                         row[7], row[8], row[12], row[13], row[16]))

<br>

Check the number of rows in combined events csv file.

In [ ]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

<br>

Create cluster and session connection.

In [ ]:
try: 
    cluster = Cluster(['cassandra_db'])
    session = cluster.connect()
except Exception as e:
    print(e)

<br>

Create and set keyspace.

In [ ]:
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS songifyks 
        WITH REPLICATION = 
        {'class': 'SimpleStrategy', 'replication_factor': 1}
    """)

except Exception as e:
    print(e)


session.set_keyspace('songifyks')

<br>

### Query 1

`artist`, `song` and `length` for `sessionId=338` and `itemInSession=4`

In [ ]:
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS songinfo_by_session_by_item (
            sessionId INT,
            itemInSession INT,
            artist TEXT,
            song TEXT,
            length FLOAT,
            PRIMARY KEY (sessionId, itemInSession));
    """)
except Exception as e:
    print(e)

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    
    for line in csvreader:
        query = "INSERT INTO songinfo_by_session_by_item (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s);"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [ ]:
session.row_factory = pandas_factory

result = session.execute("""
    SELECT artist, song, length
    FROM songinfo_by_session_by_item
    WHERE
        sessionId=338 AND
        itemInSession=4;
""")

df = result._current_rows 
df

<br>

At `sessionId=338` and `itemInsession=4`, the artist was 'Faithless', and the song was 'Music Matters (Mark Knight Dub)' with a length of 495.307312 seconds.

<br>

### Query 2

`artist`, `song` (sorted by `itemInSession`) and user (`firstName` and `lastName`) for `userid=10` and `sessionid=182`

<br>

### Query 3

all users (`firstName` and `lastName`) who listened to the `song` 'All Hands Against His Own'

<br>
Valid answer: 
Three users listened to the song 'All Hands Against His Own': Jacqueline Lynch, Sara Johnson, and Tegan Levine.

<br>

Drop the tables before closing out the sessions.

In [ ]:
try:
    session.execute("DROP TABLE songinfo_by_session_by_item")
    session.execute("DROP TABLE songinfo_by_user_by_session")
    session.execute("DROP TABLE userinfo_by_song")
except Exception as e:
    print(e)

<br>

Close the session and cluster connection.

In [ ]:
session.shutdown()
cluster.shutdown()